# How to use the RNN Autoencoder with NILMTK

This is an example on how to train and use the Recurrent Network (RNN) disaggregator on the [REDD](http://redd.csail.mit.edu/) dataset using [NILMTK](https://github.com/nilmtk/NILMTK/).

This network was described in the [Neural NILM](https://arxiv.org/pdf/1507.06594.pdf) paper.

First of all, we need to train the RNNDisaggregator using the train data. For this example, both train and test data are consumption data of the microwave of the first REDD building.

In [2]:
import warnings; warnings.simplefilter('ignore')

from nilmtk import DataSet
train = DataSet(r'C:\Users\micki\nilmtk_test\data\converted_v14.h5') #new version of train file 
train.set_window(end="2021-10-03") #because 2021-10-06 starts house1 
# from test19 file 
# TIMESTAMP_COLUMN_NAME = "timestamp"
# TIMEZONE = "Europe/London"
# START_DATETIME, END_DATETIME = '2021-09-30', '2021-10-05'
train_elec = train.buildings[1].elec

Next, we need to define the disaggregator model.

In [3]:
from rnndisaggregator import RNNDisaggregator
rnn = RNNDisaggregator()

In [4]:
from nilmtk import DataSet

sel = DataSet(r'C:\Users\micki\nilmtk_test\data\converted_v14.h5')
elec = sel.buildings[1].elec
elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='SEL', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='SEL', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=3, building=1, dataset='SEL', appliances=[Appliance(type='electric oven', instance=1)])
  ElecMeter(instance=4, building=1, dataset='SEL', appliances=[Appliance(type='kettle', instance=1)])
  ElecMeter(instance=5, building=1, dataset='SEL', appliances=[Appliance(type='stove', instance=1)])
)

In [4]:
#

Then train the model. We need to input the train data as well as their sample period. Also, we need to pass the desired number of training epochs. Finally, save the model for later use.

In [5]:
train_mains = train_elec.mains() # The aggregated meter that provides the input
train_meter = train_elec.submeters()['stove'] # The appliance meter that is used as a training target

rnn.train(train_mains, train_meter, epochs=7, sample_period=1)
rnn.export_model("model-sel_v12.h5")

Epoch 1/7
1997/1997 [==============================] - 46s 23ms/step - loss: 7.1368e-05
Epoch 2/7
1997/1997 [==============================] - 44s 22ms/step - loss: 7.0259e-05
Epoch 3/7
1997/1997 [==============================] - 43s 22ms/step - loss: 7.0278e-05
Epoch 4/7
1997/1997 [==============================] - 43s 22ms/step - loss: 7.0304e-05
Epoch 5/7
1997/1997 [==============================] - 43s 21ms/step - loss: 7.0359e-05
Epoch 6/7
1997/1997 [==============================] - 44s 22ms/step - loss: 7.0271e-05
Epoch 7/7
1997/1997 [==============================] - 48s 24ms/step - loss: 7.0321e-05


Now that the model is trained, we can use it to disaggregate energy data. Let's test it on the rest of the data from building 1.

First we use the model to predict the microwave consumption. The results are saved automatically in a .h5 datastore.

In [6]:
test = DataSet(r'C:\Users\micki\nilmtk_test\data\converted_v14.h5')
test.set_window(start="2021-09-30", end="2021-10-05")
test_elec = test.buildings[1].elec
test_mains = test_elec.mains()

disag_filename = 'disag-outSEL_v12.h5' # The filename of the resulting datastore
from nilmtk.datastore import HDFDataStore
output = HDFDataStore(disag_filename, 'w') # here you change to 'a' or 'w' depending on case 

# test_mains: The aggregated signal meter
# output: The output datastore
# train_meter: This is used in order to copy the metadata of the train meter into the datastore
rnn.disaggregate(test_mains, output, train_meter, sample_period=1)

New sensible chunk: 428401
